# Performance evaluation

### Model list: [NPE, RNPE, ABC, NPE-RS, ABC-RS]

### Simulator list: [Ricker, OUP, Turin]

### Metric list: [RMSE, MMD with temporal moment-based kernel]

In [1]:
import utils.metrics as metrics
import numpy as np
import torch
import io
import pickle
import matplotlib.pyplot as plt

from simulators.oup import oup
from simulators.ricker import ricker
from simulators.turin import turin
import warnings
warnings.filterwarnings('ignore')
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
%load_ext autoreload
%autoreload 2

cpu


In [2]:
# Utility functions

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

def load_models(root_name: str, device: torch.device):
    sum_net = torch.load("{root_name}/sum_net.pkl".format(root_name=root_name), map_location=device)

    density_estimator = torch.load("{root_name}/density_estimator.pkl".format(root_name=root_name), map_location=device)

    with open("{root_name}/posterior.pkl".format(root_name=root_name), "rb") as handle:
        posterior = CPU_Unpickler(handle).load() if device == torch.device('cpu') else pickle.load(handle)
    
    return sum_net, density_estimator, posterior


def read_rnpe(model="oup", var=1.0, degree=0, seed=1, theta=[0.5,1.0]):
    file = f"seed={seed}_degree={degree if float(degree)!=0 else 0}_var={var}_theta={theta}_{model}"

    results_dir = f"objects/RNPE/{model}_final"
    fn = f"{results_dir}/{file}.pickle"

    with open(fn, "rb") as f:
        results = pickle.load(f)

    return np.array(results['posterior_samples']['RNPE'])


def sample_posteriors(posterior, obs, num):
    return posterior.sample((num,), x=obs.reshape(1, 1, 100, -1), show_progress_bars=False)


def temporalMomentsGeneral(Y, K=3, B=4e9):
    N, Ns = Y.shape
    tau = np.linspace(0, 100, Ns)
    out = np.zeros((N, K))
    Y = Y.detach().numpy()
    for k in range(K):
        for i in range(N):
            out[i, k] = np.trapz(tau**(k) * Y[i], tau) + 1e-4
    return np.log(out)

def RMSE(gt, samples, p=1):
    if p == 1:
        dist = torch.mean(torch.abs(gt-samples))
    elif p == 2:
        dist = torch.sqrt(torch.mean((gt-samples)**2))
    elif p == 3:
        dist = torch.nn.functional.pairwise_distance(gt, samples, p=2).mean()
    else:
        dist = metrics.MMD_unweighted(samples, gt.reshape(-1, 2), lengthscale=metrics.median_heuristic(samples))
    return dist

# 1. Setup Lambda for NPE-RS and ABC-RS

## OUP

In [94]:
n_realizations_oup = 100
n_length_oup = 25

n_samples_mmd = 1000
n_sim = 50

simulator = oup(N=1)

#### NPE-RS

In [95]:
degree_list = [0.0, 0.1, 0.2]
lam_list = [1.0, 2.0, 3.0]


for degree in degree_list:
    obs = torch.tensor(np.load(f"data/oup_obs_{int(degree * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_oup, n_length_oup)))
    mmd_avg = np.zeros(len(lam_list))
    for k, lam in enumerate(lam_list):        
        mmd = np.zeros(n_sim)
        for i in range(0, n_sim):
            root_name = f"objects/NPE/oup_new/degree={degree}_var=1.0_mmd_beta={lam}_theta=[0.5, 1.0]_num=1000_N=100/{i+1}"
            _, _, posterior = load_models(root_name, device)
            post_samples = sample_posteriors(posterior, obs, n_samples_mmd)
            
            predictive_data = torch.zeros(n_samples_mmd, n_length_oup)
            for j in range(n_samples_mmd):
                predictive_data[j] = simulator(post_samples[j])[0]
    
            pred_stat = torch.tensor(temporalMomentsGeneral(predictive_data))
    
            mmd[i] = float(metrics.MMD_unweighted(pred_stat, obs_stat, 
                           lengthscale=1))
        mmd_avg[k] = np.mean(mmd[~np.isnan(mmd)])
        print(f"OUP corrupted degree={degree}, NPE-RS with lambda={lam}: MMD={mmd_avg[k]}")
    print(f"Best lambda for OUP on NPE-RS with corruption degree {degree}: {lam_list[np.argmin(mmd_avg)]}")

OUP corrupted degree=0.0, NPE-RS with lambda=1.0: MMD=0.012104899367684203
OUP corrupted degree=0.0, NPE-RS with lambda=2.0: MMD=0.04322400392710015
OUP corrupted degree=0.0, NPE-RS with lambda=3.0: MMD=0.030958189900746714
Best lambda for OUP on NPE-RS with corruption degree 0.0: 1.0
OUP corrupted degree=0.1, NPE-RS with lambda=1.0: MMD=0.08601572579174467
OUP corrupted degree=0.1, NPE-RS with lambda=2.0: MMD=0.11768026783441352
OUP corrupted degree=0.1, NPE-RS with lambda=3.0: MMD=0.15219836343072746
Best lambda for OUP on NPE-RS with corruption degree 0.1: 1.0
OUP corrupted degree=0.2, NPE-RS with lambda=1.0: MMD=0.19965814160283255
OUP corrupted degree=0.2, NPE-RS with lambda=2.0: MMD=0.16163972089600645
OUP corrupted degree=0.2, NPE-RS with lambda=3.0: MMD=0.18946449804597806
Best lambda for OUP on NPE-RS with corruption degree 0.2: 2.0


#### ABC-RS

In [90]:
degree_list = [0.0, 0.1, 0.2]
lam_list = [1, 2, 3, 4, 5, 10]

for degree in degree_list:
    obs = torch.tensor(np.load(f"data/oup_obs_{int(degree * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_oup, n_length_oup)))
    mmd_avg = np.zeros(len(lam_list))
    for k, lam in enumerate(lam_list):        
        mmd = np.zeros(n_sim)
        for i in range(n_sim):
            root_name = f"objects/ABC/oup/degree={degree}/lambda={lam}/{i}"
            post_samples = np.load(root_name + '/posterior_robust.npy')
            post_samples = torch.tensor(post_samples[np.random.choice(post_samples.shape[0], n_samples_mmd)])
            
            predictive_data = torch.zeros(n_samples_mmd, n_length_oup)
            for j in range(n_samples_mmd):
                predictive_data[j] = simulator(post_samples[j])[0]
    
            pred_stat = torch.tensor(temporalMomentsGeneral(predictive_data))
    
            mmd[i] = float(metrics.MMD_unweighted(pred_stat, obs_stat, 
                           lengthscale=1))
        mmd_avg[k] = np.mean(mmd[~np.isnan(mmd)])
        print(f"OUP corrupted degree={degree}, ABC-RS with lambda={lam}: MMD={mmd_avg[k]}")
    print(f"Best lambda for OUP on ABC-RS with corruption degree {degree}: {lam_list[np.argmin(mmd_avg)]}")

OUP corrupted degree=0.0, ABC-RS with lambda=1: MMD=0.04348598569566878
OUP corrupted degree=0.0, ABC-RS with lambda=2: MMD=0.03706016202911054
OUP corrupted degree=0.0, ABC-RS with lambda=3: MMD=0.027946326200683068
OUP corrupted degree=0.0, ABC-RS with lambda=4: MMD=0.028015741357104638
OUP corrupted degree=0.0, ABC-RS with lambda=5: MMD=0.026910244729530648
OUP corrupted degree=0.0, ABC-RS with lambda=10: MMD=0.033555406890993386
Best lambda for OUP on ABC-RS with corruption degree 0.0: 5
OUP corrupted degree=0.1, ABC-RS with lambda=1: MMD=0.7241154007271727
OUP corrupted degree=0.1, ABC-RS with lambda=2: MMD=0.6378913970338096
OUP corrupted degree=0.1, ABC-RS with lambda=3: MMD=0.4864801947605482
OUP corrupted degree=0.1, ABC-RS with lambda=4: MMD=0.45007464976722517
OUP corrupted degree=0.1, ABC-RS with lambda=5: MMD=0.416799328506095
OUP corrupted degree=0.1, ABC-RS with lambda=10: MMD=0.35603762421384066
Best lambda for OUP on ABC-RS with corruption degree 0.1: 10
OUP corrupted 

## Ricker

In [3]:
n_realizations_ricker = 100
n_length_ricker = 100

n_samples_mmd = 1000
n_sim = 50

simulator = ricker(N=1)

#### NPE-RS

In [30]:
degree_list = [0.0, 0.1, 0.2]
lam_list = [1.0, 2.0, 3.0, 5.0, 10.0]


for degree in degree_list:
    obs = torch.tensor(np.load(f"data/ricker_obs_mix_{int(float(degree) * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_ricker, n_length_ricker)))
    mmd_avg = np.zeros(len(lam_list))
    for k, lam in enumerate(lam_list):        
        mmd = np.zeros(n_sim)
        for i in range(0, n_sim):
            root_name = f"objects/NPE/ricker/mix/degree={degree}_mmd_beta={lam}_theta=[4, 10]_num=1000/{i+1}"
            _, _, posterior = load_models(root_name, device)
            post_samples = sample_posteriors(posterior, obs, n_samples_mmd)
            
            predictive_data = torch.zeros(n_samples_mmd, n_length_ricker)
            for j in range(n_samples_mmd):
                predictive_data[j] = simulator(post_samples[j])[0]
    
            pred_stat = torch.tensor(temporalMomentsGeneral(predictive_data))
    
            mmd[i] = float(metrics.MMD_unweighted(pred_stat, obs_stat, 
                           lengthscale=1))
        mmd_avg[k] = np.mean(mmd[~np.isnan(mmd)])
        print(f"Ricker corrupted degree={degree}, NPE-RS with lambda={lam}: MMD={mmd_avg[k]}")
    print(f"Best lambda for Ricker on NPE-RS with corruption degree {degree}: {lam_list[np.argmin(mmd_avg)]}")

Ricker corrupted degree=0.0, NPE-RS with lambda=1.0: MMD=0.08868219585875839
Ricker corrupted degree=0.0, NPE-RS with lambda=2.0: MMD=0.10531050150119718
Ricker corrupted degree=0.0, NPE-RS with lambda=3.0: MMD=0.16048733856418285
Ricker corrupted degree=0.0, NPE-RS with lambda=5.0: MMD=0.20432387258646473
Ricker corrupted degree=0.0, NPE-RS with lambda=10.0: MMD=0.2818680573878765
Best lambda for Ricker on NPE-RS with corruption degree 0.0: 1.0
Ricker corrupted degree=0.1, NPE-RS with lambda=1.0: MMD=0.28642005065121234
Ricker corrupted degree=0.1, NPE-RS with lambda=2.0: MMD=0.2270608896447657
Ricker corrupted degree=0.1, NPE-RS with lambda=3.0: MMD=0.30535392775018233
Ricker corrupted degree=0.1, NPE-RS with lambda=5.0: MMD=0.3268486259108123
Ricker corrupted degree=0.1, NPE-RS with lambda=10.0: MMD=0.38814205849332295
Best lambda for Ricker on NPE-RS with corruption degree 0.1: 2.0
Ricker corrupted degree=0.2, NPE-RS with lambda=1.0: MMD=0.3115958120943685
Ricker corrupted degree=0

#### ABC-RS

In [29]:
degree_list = [0.0, 0.1, 0.2]
lam_list = [1, 2, 3, 4, 5, 10]

for degree in degree_list:
    obs = torch.tensor(np.load(f"data/ricker_obs_mix_{int(degree * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_ricker, n_length_ricker)))
    mmd_avg = np.zeros(len(lam_list))
    for k, lam in enumerate(lam_list):        
        mmd = np.zeros(n_sim)
        for i in range(n_sim):
            root_name = f"objects/ABC/ricker/degree={degree}/lambda={lam}/{i}"
            post_samples = np.load(root_name + '/posterior_robust.npy')
            post_samples = torch.tensor(post_samples[np.random.choice(post_samples.shape[0], n_samples_mmd)])
            
            predictive_data = torch.zeros(n_samples_mmd, n_length_ricker)
            for j in range(n_samples_mmd):
                predictive_data[j] = simulator(post_samples[j])[0]
    
            pred_stat = torch.tensor(temporalMomentsGeneral(predictive_data))
    
            mmd[i] = float(metrics.MMD_unweighted(pred_stat, obs_stat, 
                           lengthscale=1))
        mmd_avg[k] = np.mean(mmd[~np.isnan(mmd)])
        print(f"Ricker corrupted degree={degree}, ABC-RS with lambda={lam}: MMD={mmd_avg[k]}")
    print(f"Best lambda for Ricker on ABC-RS with corruption degree {degree}: {lam_list[np.argmin(mmd_avg)]}")

Ricker corrupted degree=0.0, ABC-RS with lambda=1: MMD=0.005767172948180837
Ricker corrupted degree=0.0, ABC-RS with lambda=2: MMD=0.005728841590774645
Ricker corrupted degree=0.0, ABC-RS with lambda=3: MMD=0.014609483353472811
Ricker corrupted degree=0.0, ABC-RS with lambda=4: MMD=0.02558981398591236
Ricker corrupted degree=0.0, ABC-RS with lambda=5: MMD=0.02946696851842721
Ricker corrupted degree=0.0, ABC-RS with lambda=10: MMD=0.06533276035192817
Best lambda for Ricker on ABC-RS with corruption degree 0.0: 2
Ricker corrupted degree=0.1, ABC-RS with lambda=1: MMD=0.2718213878047113
Ricker corrupted degree=0.1, ABC-RS with lambda=2: MMD=0.2166386489705339
Ricker corrupted degree=0.1, ABC-RS with lambda=3: MMD=0.19601454058791679
Ricker corrupted degree=0.1, ABC-RS with lambda=4: MMD=0.19796378541784315
Ricker corrupted degree=0.1, ABC-RS with lambda=5: MMD=0.1560059810327139
Ricker corrupted degree=0.1, ABC-RS with lambda=10: MMD=0.18245332968158234
Best lambda for Ricker on ABC-RS wi

# 2. Final evaluation

## OUP

In [3]:
n_realizations_oup = 100
n_length_oup = 25

n_samples_mmd = 1000
n_samples_rmse = 1000

n_sim = 100

simulator = oup(N=1)

theta_gt = torch.tensor([0.5, 1.0])

### OUP RMSE (NPE family)

In [145]:
degree_lam_npe_rs = {'0.0': 1.0, '0.1': 1.0, '0.2': 2.0}

oup_rmse_npe = np.zeros((3, 2))
oup_rmse_rnpe = np.zeros((3, 2))
oup_rmse_npe_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_npe_rs.items()):
    obs = torch.tensor(np.load(f"data/oup_obs_{int(float(degree) * 10)}.npy"))
    
    rmse_npe = np.zeros(n_sim)
    rmse_rnpe = np.zeros(n_sim)
    rmse_npe_rs = np.zeros(n_sim)
    for i in range(0, n_sim):   
        root_name = f'objects/NPE/oup_final/degree=0.0_var=1.0_none_beta=3.0_theta=[0.5, 1.0]_num=1000_N=100/{i+1}'
        _, _, posterior_npe = load_models(root_name, device)
        post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_rmse)
        
        root_name = f'objects/NPE/oup_final/degree={degree}_var=1.0_mmd_beta={lam}_theta=[0.5, 1.0]_num=1000_N=100/{i+1}'
        _, _, posterior_npe_rs = load_models(root_name, device)
        post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_rmse)
        
        post_samples_rnpe = torch.tensor(read_rnpe(model="oup", var=1, degree=degree, seed=i+1, theta='[0.5,1.0]'))[:n_samples_rmse]
        
        
        rmse_npe[i] = float(RMSE(theta_gt, post_samples_npe, p=2))
        rmse_rnpe[i] = float(RMSE(theta_gt, post_samples_rnpe, p=2))
        rmse_npe_rs[i] = float(RMSE(theta_gt, post_samples_npe_rs, p=2))
                               
    oup_rmse_npe[k, 0] = np.mean(rmse_npe)
    oup_rmse_npe[k, 1] = np.std(rmse_npe)
    
    oup_rmse_rnpe[k, 0] = np.mean(rmse_rnpe)
    oup_rmse_rnpe[k, 1] = np.std(rmse_rnpe)
                               
    oup_rmse_npe_rs[k, 0] = np.mean(rmse_npe_rs)
    oup_rmse_npe_rs[k, 1] = np.std(rmse_npe_rs)
    
    print(f"OUP corrupted degree={degree}")
    print(f"NPE: RMSE mean={oup_rmse_npe[k, 0]}, RMSE std={oup_rmse_npe[k, 1]}")
    print(f"RNPE: RMSE mean={oup_rmse_rnpe[k, 0]}, RMSE std={oup_rmse_rnpe[k, 1]}")
    print(f"NPE-RS with lambda={lam}: RMSE mean={oup_rmse_npe_rs[k, 0]}, RMSE std={oup_rmse_npe_rs[k, 1]}")

OUP corrupted degree=0.0
NPE: RMSE mean=0.7925630547851324, RMSE std=0.6189539155079293
RNPE: RMSE mean=0.7798082494735717, RMSE std=0.08754678141313765
NPE-RS with lambda=1.0: RMSE mean=0.7372859045863152, RMSE std=0.6026562181259022
OUP corrupted degree=0.1
NPE: RMSE mean=1.2649758909642697, RMSE std=1.1900491524141108
RNPE: RMSE mean=0.8717409908771515, RMSE std=0.10160560064131247
NPE-RS with lambda=1.0: RMSE mean=0.6163115787506104, RMSE std=0.3190021981676373
OUP corrupted degree=0.2
NPE: RMSE mean=2.5887098464369775, RMSE std=2.7487078421648006
RNPE: RMSE mean=0.9750068730115891, RMSE std=0.15094306281805106
NPE-RS with lambda=2.0: RMSE mean=0.6315770514309407, RMSE std=0.35337456682942775


### OUP RMSE (ABC family)

We nned to do ABC separately as it use different set of lambdas

In [10]:
degree_lam_abc_rs = {'0.0': 5, '0.1': 15, '0.2': 20}

oup_rmse_abc = np.zeros((3, 2))
oup_rmse_abc_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_abc_rs.items()):
    obs = torch.tensor(np.load(f"data/oup_obs_{int(float(degree) * 10)}.npy"))
       
    rmse_abc = np.zeros(n_sim)
    rmse_abc_rs = np.zeros(n_sim)
    for i in range(0, n_sim):
        # share the root name
        root_name = f"objects/ABC/oup_final/degree={degree}/lambda={lam}/{i}"
        
        # abc
        post_samples_abc = np.load(root_name + '/posterior_normal.npy')
        post_samples_abc = torch.tensor(post_samples_abc[np.random.choice(post_samples_abc.shape[0], n_samples_rmse)])

        # abc_rs
        post_samples_abc_rs = np.load(root_name + '/posterior_robust.npy')
        post_samples_abc_rs = torch.tensor(post_samples_abc_rs[np.random.choice(post_samples_abc_rs.shape[0], n_samples_rmse)])

        rmse_abc[i] = float(RMSE(theta_gt, post_samples_abc, p=2))
        rmse_abc_rs[i] = float(RMSE(theta_gt, post_samples_abc_rs, p=2))
        
    oup_rmse_abc[k, 0] = np.mean(rmse_abc)
    oup_rmse_abc[k, 1] = np.std(rmse_abc)
    
    oup_rmse_abc_rs[k, 0] = np.mean(rmse_abc_rs)
    oup_rmse_abc_rs[k, 1] = np.std(rmse_abc_rs)
    
    print(f"OUP corrupted degree={degree}")
    print(f"ABC: RMSE mean={oup_rmse_abc[k, 0]}, RMSE std={oup_rmse_abc[k, 1]}")
    print(f"ABC-RS with lambda={lam}: RMSE mean={oup_rmse_abc_rs[k, 0]}, RMSE std={oup_rmse_abc_rs[k, 1]}")


OUP corrupted degree=0.0
ABC: RMSE mean=0.49509559582020657, RMSE std=0.06467575409620172
ABC-RS with lambda=5: RMSE mean=0.4418196542316118, RMSE std=0.06685935044476349
OUP corrupted degree=0.1
ABC: RMSE mean=1.2996316719499046, RMSE std=0.3686765148306931
ABC-RS with lambda=15: RMSE mean=0.6165489532625812, RMSE std=0.22824126181335438
OUP corrupted degree=0.2
ABC: RMSE mean=5.370637510496948, RMSE std=2.3427128599919267
ABC-RS with lambda=20: RMSE mean=0.8767756478055136, RMSE std=0.4821035779200563


### OUP MMD (NPE family)

In [146]:
degree_lam_npe_rs = {'0.0':1.0, '0.1':1.0, '0.2': 2.0}

oup_mmd_npe = np.zeros((3, 2))
oup_mmd_rnpe = np.zeros((3, 2))
oup_mmd_npe_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_npe_rs.items()):
    obs = torch.tensor(np.load(f"data/oup_obs_{int(float(degree) * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_oup, n_length_oup)))
       
    mmd_npe = np.zeros(n_sim)
    mmd_rnpe = np.zeros(n_sim)
    mmd_npe_rs = np.zeros(n_sim)
    for i in range(0, n_sim):
        # npe
        root_name = f"objects/NPE/oup_final/degree=0.0_var=1.0_none_beta=3.0_theta=[0.5, 1.0]_num=1000_N=100/{i+1}"
        _, _, posterior_npe = load_models(root_name, device)
        post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_mmd)
        
        predictive_data_npe = torch.zeros(n_samples_mmd, n_length_oup)
        for j in range(n_samples_mmd):
            predictive_data_npe[j] = simulator(post_samples_npe[j])[0]

        pred_stat_npe = torch.tensor(temporalMomentsGeneral(predictive_data_npe))

        mmd_npe[i] = float(metrics.MMD_unweighted(pred_stat_npe, obs_stat, lengthscale=1))
        
        # rnpe
        post_samples_rnpe = torch.tensor(read_rnpe(model="oup", var=1, degree=degree, seed=i+1, theta='[0.5,1.0]'))[:n_samples_mmd]

        predictive_data_rnpe = torch.zeros(n_samples_mmd, n_length_oup)
        for j in range(n_samples_mmd):
            predictive_data_rnpe[j] = simulator(post_samples_rnpe[j])[0]

        pred_stat_rnpe = torch.tensor(temporalMomentsGeneral(predictive_data_rnpe))

        mmd_rnpe[i] = float(metrics.MMD_unweighted(pred_stat_rnpe, obs_stat, lengthscale=1))
        
        # npe_rs
        root_name = f"objects/NPE/oup_final/degree={degree}_var=1.0_mmd_beta={lam}_theta=[0.5, 1.0]_num=1000_N=100/{i+1}"
        _, _, posterior_npe_rs = load_models(root_name, device)
        post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_mmd)

        predictive_data_npe_rs = torch.zeros(n_samples_mmd, n_length_oup)
        for j in range(n_samples_mmd):
            predictive_data_npe_rs[j] = simulator(post_samples_npe_rs[j])[0]

        pred_stat_npe_rs = torch.tensor(temporalMomentsGeneral(predictive_data_npe_rs))

        mmd_npe_rs[i] = float(metrics.MMD_unweighted(pred_stat_npe_rs, obs_stat, lengthscale=1))
        
    oup_mmd_npe[k, 0] = np.mean(mmd_npe[~np.isnan(mmd_npe)])
    oup_mmd_npe[k, 1] = np.std(mmd_npe[~np.isnan(mmd_npe)])
    
    oup_mmd_rnpe[k, 0] = np.mean(mmd_rnpe[~np.isnan(mmd_rnpe)])
    oup_mmd_rnpe[k, 1] = np.std(mmd_rnpe[~np.isnan(mmd_rnpe)])
    
    oup_mmd_npe_rs[k, 0] = np.mean(mmd_npe_rs[~np.isnan(mmd_npe_rs)])
    oup_mmd_npe_rs[k, 1] = np.std(mmd_npe_rs[~np.isnan(mmd_npe_rs)])
    
    print(f"OUP corrupted degree={degree}")
    print(f"NPE: MMD mean={oup_mmd_npe[k, 0]}, MMD std={oup_mmd_npe[k, 1]}")
    print(f"RNPE: MMD mean={oup_mmd_rnpe[k, 0]}, MMD std={oup_mmd_rnpe[k, 1]}")
    print(f"NPE-RS with lambda={lam}: MMD mean={oup_mmd_npe_rs[k, 0]}, MMD std={oup_mmd_npe_rs[k, 1]}")


OUP corrupted degree=0.0
NPE: MMD mean=0.0070277369812554585, MMD std=0.010243516876271171
RNPE: MMD mean=0.011334007263055793, MMD std=0.009915834978669363
NPE-RS with lambda=1.0: MMD mean=0.022377721632277688, MMD std=0.04952137444403441
OUP corrupted degree=0.1
NPE: MMD mean=0.3393863778075364, MMD std=0.14622658405849778
RNPE: MMD mean=0.21543395248179795, MMD std=0.1259491349758079
NPE-RS with lambda=1.0: MMD mean=0.08563636728201396, MMD std=0.09430481767122267
OUP corrupted degree=0.2
NPE: MMD mean=0.6459693757398669, MMD std=0.2916390504590604
RNPE: MMD mean=0.48738906279377814, MMD std=0.2608236681747754
NPE-RS with lambda=2.0: MMD mean=0.20954411375210164, MMD std=0.1700419374910337


### OUP MMD (ABC family)

In [11]:
degree_lam_abc_rs = {'0.0': 5, '0.1': 15, '0.2': 20}

oup_mmd_abc = np.zeros((3, 2))
oup_mmd_abc_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_abc_rs.items()):
    obs = torch.tensor(np.load(f"data/oup_obs_{int(float(degree) * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_oup, n_length_oup)))
       
    mmd_abc = np.zeros(n_sim)
    mmd_abc_rs = np.zeros(n_sim)
    for i in range(0, n_sim):
        # share the root name
        root_name = f"objects/ABC/oup_final/degree={degree}/lambda={lam}/{i}"
        
        # abc
        post_samples_abc = np.load(root_name + '/posterior_normal.npy')
        post_samples_abc = torch.tensor(post_samples_abc[np.random.choice(post_samples_abc.shape[0], n_samples_mmd)])

        predictive_data_abc = torch.zeros(n_samples_mmd, n_length_oup)
        for j in range(n_samples_mmd):
            predictive_data_abc[j] = simulator(post_samples_abc[j])[0]

        pred_stat_abc = torch.tensor(temporalMomentsGeneral(predictive_data_abc))

        mmd_abc[i] = float(metrics.MMD_unweighted(pred_stat_abc, obs_stat, 
                              lengthscale=1))
        
        # abc_rs
        post_samples_abc_rs = np.load(root_name + '/posterior_robust.npy')
        post_samples_abc_rs = torch.tensor(post_samples_abc_rs[np.random.choice(post_samples_abc_rs.shape[0], n_samples_mmd)])

        predictive_data_abc_rs = torch.zeros(n_samples_mmd, n_length_oup)
        for j in range(n_samples_mmd):
            predictive_data_abc_rs[j] = simulator(post_samples_abc_rs[j])[0]

        pred_stat_abc_rs = torch.tensor(temporalMomentsGeneral(predictive_data_abc_rs))

        mmd_abc_rs[i] = float(metrics.MMD_unweighted(pred_stat_abc_rs, obs_stat, 
                              lengthscale=1))
        
    oup_mmd_abc[k, 0] = np.mean(mmd_abc[~np.isnan(mmd_abc)])
    oup_mmd_abc[k, 1] = np.std(mmd_abc[~np.isnan(mmd_abc)])
    
    oup_mmd_abc_rs[k, 0] = np.mean(mmd_abc_rs[~np.isnan(mmd_abc_rs)])
    oup_mmd_abc_rs[k, 1] = np.std(mmd_abc_rs[~np.isnan(mmd_abc_rs)])
    
    print(f"OUP corrupted degree={degree}")
    print(f"ABC: MMD mean={oup_mmd_abc[k, 0]}, MMD std={oup_mmd_abc[k, 1]}")
    print(f"ABC-RS with lambda={lam}: MMD mean={oup_mmd_abc_rs[k, 0]}, MMD std={oup_mmd_abc_rs[k, 1]}")


OUP corrupted degree=0.0
ABC: MMD mean=0.051694323171523386, MMD std=0.030256493560625913
ABC-RS with lambda=5: MMD mean=0.02326028402970792, MMD std=0.020645582131445087
OUP corrupted degree=0.1
ABC: MMD mean=0.9074055313005361, MMD std=0.2066785989071141
ABC-RS with lambda=15: MMD mean=0.26036175151166246, MMD std=0.16959001923327374
OUP corrupted degree=0.2
ABC: MMD mean=0.9359434314371308, MMD std=0.19424011524359938
ABC-RS with lambda=20: MMD mean=0.49584723062468256, MMD std=0.37667546112109673


## Ricker

In [50]:
n_realizations_ricker = 100
n_length_ricker = 100

n_samples_mmd = 1000
n_samples_rmse = 1000
n_sim = 100

simulator = ricker(N=1)

theta_gt_ricker = torch.tensor([4, 10])

### Ricker RMSE (NPE family)

In [5]:
degree_lam_npe_rs = {'0.0': 1.0, '0.1': 2.0, '0.2': 1.0}

ricker_rmse_npe = np.zeros((3, 2))
ricker_rmse_rnpe = np.zeros((3, 2))
ricker_rmse_npe_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_npe_rs.items()):
    obs = torch.tensor(np.load(f"data/ricker_obs_mix_{int(float(degree) * 10)}.npy"))
    
    rmse_npe = np.zeros(n_sim)
    rmse_rnpe = np.zeros(n_sim)
    rmse_npe_rs = np.zeros(n_sim)
    for i in range(0, n_sim):   
        root_name = f'objects/NPE/ricker_final/degree=0.0_none_beta=1.0_theta=[4, 10]_num=1000/{i+1}'
        _, _, posterior_npe = load_models(root_name, device)
        post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_rmse)
        
        root_name = f'objects/NPE/ricker_final/degree={degree}_mmd_beta={lam}_theta=[4, 10]_num=1000/{i+1}'
        _, _, posterior_npe_rs = load_models(root_name, device)
        post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_rmse)
        
        post_samples_rnpe = torch.tensor(read_rnpe(model="ricker", var=1, degree=degree, seed=i+1, theta='[4,10]'))[:n_samples_rmse]
        
        
        rmse_npe[i] = float(RMSE(theta_gt_ricker, post_samples_npe, p=2))
        rmse_rnpe[i] = float(RMSE(theta_gt_ricker, post_samples_rnpe, p=2))
        rmse_npe_rs[i] = float(RMSE(theta_gt_ricker, post_samples_npe_rs, p=2))
                               
    ricker_rmse_npe[k, 0] = np.mean(rmse_npe)
    ricker_rmse_npe[k, 1] = np.std(rmse_npe)
    
    ricker_rmse_rnpe[k, 0] = np.mean(rmse_rnpe)
    ricker_rmse_rnpe[k, 1] = np.std(rmse_rnpe)
                               
    ricker_rmse_npe_rs[k, 0] = np.mean(rmse_npe_rs)
    ricker_rmse_npe_rs[k, 1] = np.std(rmse_npe_rs)
    
    print(f"Ricker corrupted degree={degree}")
    print(f"NPE: RMSE mean={ricker_rmse_npe[k, 0]}, RMSE std={ricker_rmse_npe[k, 1]}")
    print(f"RNPE: RMSE mean={ricker_rmse_rnpe[k, 0]}, RMSE std={ricker_rmse_rnpe[k, 1]}")
    print(f"NPE-RS with lambda={lam}: RMSE mean={ricker_rmse_npe_rs[k, 0]}, RMSE std={ricker_rmse_npe_rs[k, 1]}")

OUP corrupted degree=0.0
NPE: RMSE mean=2.159665313065052, RMSE std=3.072399608646757
RNPE: RMSE mean=3.272554533481598, RMSE std=0.3458116151400395
NPE-RS with lambda=1.0: RMSE mean=2.186364681571722, RMSE std=2.6600947351637276
OUP corrupted degree=0.1
NPE: RMSE mean=7.862835259437561, RMSE std=1.5652844966705644
RNPE: RMSE mean=5.505721306800842, RMSE std=0.5761808462790206
NPE-RS with lambda=2.0: RMSE mean=2.1900993782281875, RMSE std=1.0142630566053492
OUP corrupted degree=0.2
NPE: RMSE mean=11.254029388427734, RMSE std=1.6961005782948435
RNPE: RMSE mean=7.1370215463638305, RMSE std=1.1523687822492195
NPE-RS with lambda=1.0: RMSE mean=4.659657540917396, RMSE std=4.147216315165953


### Ricker RMSE (ABC family)

In [5]:
degree_lam_abc_rs = {'0.0': 2, '0.1': 5, '0.2': 3}

ricker_rmse_abc = np.zeros((3, 2))
ricker_rmse_abc_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_abc_rs.items()):
    obs = torch.tensor(np.load(f"data/ricker_obs_mix_{int(float(degree) * 10)}.npy"))
       
    rmse_abc = np.zeros(n_sim)
    rmse_abc_rs = np.zeros(n_sim)
    for i in range(0, n_sim):
        # share the root name
        root_name = f"objects/ABC/ricker_final/degree={degree}/lambda={lam}/{i}"
        
        # abc
        post_samples_abc = np.load(root_name + '/posterior_normal.npy')
        post_samples_abc = torch.tensor(post_samples_abc[np.random.choice(post_samples_abc.shape[0], n_samples_rmse)])

        # abc_rs
        post_samples_abc_rs = np.load(root_name + '/posterior_robust.npy')
        post_samples_abc_rs = torch.tensor(post_samples_abc_rs[np.random.choice(post_samples_abc_rs.shape[0], n_samples_rmse)])

        rmse_abc[i] = float(RMSE(theta_gt_ricker, post_samples_abc, p=2))
        rmse_abc_rs[i] = float(RMSE(theta_gt_ricker, post_samples_abc_rs, p=2))
        
    ricker_rmse_abc[k, 0] = np.mean(rmse_abc)
    ricker_rmse_abc[k, 1] = np.std(rmse_abc)
    
    ricker_rmse_abc_rs[k, 0] = np.mean(rmse_abc_rs)
    ricker_rmse_abc_rs[k, 1] = np.std(rmse_abc_rs)
    
    print(f"Ricker corrupted degree={degree}")
    print(f"ABC: RMSE mean={ricker_rmse_abc[k, 0]}, RMSE std={ricker_rmse_abc[k, 1]}")
    print(f"ABC-RS with lambda={lam}: RMSE mean={ricker_rmse_abc_rs[k, 0]}, RMSE std={ricker_rmse_abc_rs[k, 1]}")


Ricker corrupted degree=0.0
ABC: RMSE mean=1.4608000411195203, RMSE std=0.4392969559445925
ABC-RS with lambda=2: RMSE mean=1.2027322280237103, RMSE std=0.5087569342337235
Ricker corrupted degree=0.1
ABC: RMSE mean=6.954336906258834, RMSE std=0.24572330168065148
ABC-RS with lambda=5: RMSE mean=3.1578248301227183, RMSE std=1.0781114401159106
Ricker corrupted degree=0.2
ABC: RMSE mean=9.790066190478761, RMSE std=0.9581979221745782
ABC-RS with lambda=3: RMSE mean=2.988856295848981, RMSE std=1.2768888852086335


### Ricker MMD (NPE family)

In [6]:
degree_lam_npe_rs = {'0.0': 1.0, '0.1': 2.0, '0.2': 1.0}

ricker_mmd_npe = np.zeros((3, 2))
ricker_mmd_rnpe = np.zeros((3, 2))
ricker_mmd_npe_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_npe_rs.items()):
    obs = torch.tensor(np.load(f"data/ricker_obs_mix_{int(float(degree) * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_ricker, n_length_ricker)))
       
    mmd_npe = np.zeros(n_sim)
    mmd_rnpe = np.zeros(n_sim)
    mmd_npe_rs = np.zeros(n_sim)
    for i in range(0, n_sim):
        # npe
        root_name = f"objects/NPE/ricker_final/degree=0.0_none_beta=1.0_theta=[4, 10]_num=1000/{i+1}"
        _, _, posterior_npe = load_models(root_name, device)
        post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_mmd)
        
        predictive_data_npe = torch.zeros(n_samples_mmd, n_length_ricker)
        for j in range(n_samples_mmd):
            predictive_data_npe[j] = simulator(post_samples_npe[j])[0]

        pred_stat_npe = torch.tensor(temporalMomentsGeneral(predictive_data_npe))

        mmd_npe[i] = float(metrics.MMD_unweighted(pred_stat_npe, obs_stat, lengthscale=1))
        
        # rnpe
        post_samples_rnpe = torch.tensor(read_rnpe(model="ricker", var=1, degree=degree, seed=i+1, theta='[4,10]'))[:n_samples_mmd]

        predictive_data_rnpe = torch.zeros(n_samples_mmd, n_length_ricker)
        for j in range(n_samples_mmd):
            predictive_data_rnpe[j] = simulator(post_samples_rnpe[j])[0]

        pred_stat_rnpe = torch.tensor(temporalMomentsGeneral(predictive_data_rnpe))

        mmd_rnpe[i] = float(metrics.MMD_unweighted(pred_stat_rnpe, obs_stat, lengthscale=1))
        
        # npe_rs
        root_name = f"objects/NPE/ricker_final/degree={degree}_mmd_beta={lam}_theta=[4, 10]_num=1000/{i+1}"
        _, _, posterior_npe_rs = load_models(root_name, device)
        post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_mmd)

        predictive_data_npe_rs = torch.zeros(n_samples_mmd, n_length_ricker)
        for j in range(n_samples_mmd):
            predictive_data_npe_rs[j] = simulator(post_samples_npe_rs[j])[0]

        pred_stat_npe_rs = torch.tensor(temporalMomentsGeneral(predictive_data_npe_rs))

        mmd_npe_rs[i] = float(metrics.MMD_unweighted(pred_stat_npe_rs, obs_stat, lengthscale=1))
        
    ricker_mmd_npe[k, 0] = np.mean(mmd_npe[~np.isnan(mmd_npe)])
    ricker_mmd_npe[k, 1] = np.std(mmd_npe[~np.isnan(mmd_npe)])
    
    ricker_mmd_rnpe[k, 0] = np.mean(mmd_rnpe[~np.isnan(mmd_rnpe)])
    ricker_mmd_rnpe[k, 1] = np.std(mmd_rnpe[~np.isnan(mmd_rnpe)])
    
    ricker_mmd_npe_rs[k, 0] = np.mean(mmd_npe_rs[~np.isnan(mmd_npe_rs)])
    ricker_mmd_npe_rs[k, 1] = np.std(mmd_npe_rs[~np.isnan(mmd_npe_rs)])
    
    print(f"Ricker corrupted degree={degree}")
    print(f"NPE: MMD mean={ricker_mmd_npe[k, 0]}, MMD std={ricker_mmd_npe[k, 1]}")
    print(f"RNPE: MMD mean={ricker_mmd_rnpe[k, 0]}, MMD std={ricker_mmd_rnpe[k, 1]}")
    print(f"NPE-RS with lambda={lam}: MMD mean={ricker_mmd_npe_rs[k, 0]}, MMD std={ricker_mmd_npe_rs[k, 1]}")


OUP corrupted degree=0.0
NPE: MMD mean=0.03706105067511231, MMD std=0.06727287126878594
RNPE: MMD mean=0.060767924199361306, MMD std=0.05112879165455601
NPE-RS with lambda=1.0: MMD mean=0.09476873343865283, MMD std=0.1360528195996939
OUP corrupted degree=0.1
NPE: MMD mean=0.7407399593116799, MMD std=0.09144744914533502
RNPE: MMD mean=0.5144580338583704, MMD std=0.19024194378245304
NPE-RS with lambda=2.0: MMD mean=0.21128224107432303, MMD std=0.16095958734150315
OUP corrupted degree=0.2
NPE: MMD mean=1.062957432326659, MMD std=0.1665800264563046
RNPE: MMD mean=0.7876811902890908, MMD std=0.2516543011128707
NPE-RS with lambda=1.0: MMD mean=0.4162054194272693, MMD std=0.3729153105951972


### Ricker MMD (ABC family)

In [6]:
degree_lam_abc_rs = {'0.0': 2, '0.1': 5, '0.2': 3}

ricker_mmd_abc = np.zeros((3, 2))
ricker_mmd_abc_rs = np.zeros((3, 2))

for k, (degree, lam) in enumerate(degree_lam_abc_rs.items()):
    obs = torch.tensor(np.load(f"data/ricker_obs_mix_{int(float(degree) * 10)}.npy"))
    obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_ricker, n_length_ricker)))
       
    mmd_abc = np.zeros(n_sim)
    mmd_abc_rs = np.zeros(n_sim)
    for i in range(0, n_sim):
        # share the root name
        root_name = f"objects/ABC/ricker_final/degree={degree}/lambda={lam}/{i}"
        
        # abc
        post_samples_abc = np.load(root_name + '/posterior_normal.npy')
        post_samples_abc = torch.tensor(post_samples_abc[np.random.choice(post_samples_abc.shape[0], n_samples_mmd)])

        predictive_data_abc = torch.zeros(n_samples_mmd, n_length_ricker)
        for j in range(n_samples_mmd):
            predictive_data_abc[j] = simulator(post_samples_abc[j])[0]

        pred_stat_abc = torch.tensor(temporalMomentsGeneral(predictive_data_abc))

        mmd_abc[i] = float(metrics.MMD_unweighted(pred_stat_abc, obs_stat, 
                              lengthscale=1))
        
        # abc_rs
        post_samples_abc_rs = np.load(root_name + '/posterior_robust.npy')
        post_samples_abc_rs = torch.tensor(post_samples_abc_rs[np.random.choice(post_samples_abc_rs.shape[0], n_samples_mmd)])

        predictive_data_abc_rs = torch.zeros(n_samples_mmd, n_length_ricker)
        for j in range(n_samples_mmd):
            predictive_data_abc_rs[j] = simulator(post_samples_abc_rs[j])[0]

        pred_stat_abc_rs = torch.tensor(temporalMomentsGeneral(predictive_data_abc_rs))

        mmd_abc_rs[i] = float(metrics.MMD_unweighted(pred_stat_abc_rs, obs_stat, 
                              lengthscale=1))
        
    ricker_mmd_abc[k, 0] = np.mean(mmd_abc[~np.isnan(mmd_abc)])
    ricker_mmd_abc[k, 1] = np.std(mmd_abc[~np.isnan(mmd_abc)])
    
    ricker_mmd_abc_rs[k, 0] = np.mean(mmd_abc_rs[~np.isnan(mmd_abc_rs)])
    ricker_mmd_abc_rs[k, 1] = np.std(mmd_abc_rs[~np.isnan(mmd_abc_rs)])
    
    print(f"Ricker corrupted degree={degree}")
    print(f"ABC: MMD mean={ricker_mmd_abc[k, 0]}, MMD std={ricker_mmd_abc[k, 1]}")
    print(f"ABC-RS with lambda={lam}: MMD mean={ricker_mmd_abc_rs[k, 0]}, MMD std={ricker_mmd_abc_rs[k, 1]}")


Ricker corrupted degree=0.0
ABC: MMD mean=0.011006514798503762, MMD std=0.006379625010296103
ABC-RS with lambda=2: MMD mean=0.010748643338569562, MMD std=0.024220475619427403
Ricker corrupted degree=0.1
ABC: MMD mean=0.8466678177737447, MMD std=0.020180858060685626
ABC-RS with lambda=5: MMD mean=0.17617922205863845, MMD std=0.14967006191022045
Ricker corrupted degree=0.2
ABC: MMD mean=1.1844820523373287, MMD std=0.04169950910295297
ABC-RS with lambda=3: MMD mean=0.1798236737689743, MMD std=0.15902950406937638


### Ricker prior misspecification RMSE

In [56]:
theta_gt_ricker = torch.tensor([4, 25])
obs = torch.tensor(np.load(f"data/ricker_obs_pm.npy"))

rmse_npe = np.zeros(n_sim)
rmse_rnpe = np.zeros(n_sim)
rmse_npe_rs = np.zeros(n_sim)
for i in range(0, n_sim):   
    root_name = f'objects/NPE/ricker_final/degree=0.2_none_beta=2.0_theta=[4, 25]_num=1000/{i+1}'
    _, _, posterior_npe = load_models(root_name, device)
    post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_rmse)

    root_name = f'objects/NPE/ricker_final/degree=0.2_mmd_beta=5.0_theta=[4, 25]_num=1000/{i+1}'
    _, _, posterior_npe_rs = load_models(root_name, device)
    post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_rmse)

    post_samples_rnpe = torch.tensor(read_rnpe(model="ricker", var=1, degree=0, seed=i+1, theta='[4,25]'))[:n_samples_rmse]

    rmse_npe[i] = float(RMSE(theta_gt_ricker, post_samples_npe, p=2))
    rmse_rnpe[i] = float(RMSE(theta_gt_ricker, post_samples_rnpe, p=2))
    rmse_npe_rs[i] = float(RMSE(theta_gt_ricker, post_samples_npe_rs, p=2))
    
print(f"Ricker prior misspecification")
print(f"NPE: RMSE mean={np.mean(rmse_npe)}, RMSE std={np.std(rmse_npe)}")
print(f"RNPE: RMSE mean={np.mean(rmse_rnpe)}, RMSE std={np.std(rmse_rnpe)}")
print(f"NPE-RS: RMSE mean={np.mean(rmse_npe_rs)}, RMSE std={np.std(rmse_npe_rs)}")

Ricker prior misspecification
NPE: RMSE mean=7.719186365604401, RMSE std=4.024103435366731
RNPE: RMSE mean=7.707057628631592, RMSE std=2.8867418100218227
NPE-RS: RMSE mean=6.461816523075104, RMSE std=4.3615884296198075


### Ricker prior misspecification MMD

In [57]:
obs = torch.tensor(np.load(f"data/ricker_obs_pm.npy"))
obs_stat = torch.tensor(temporalMomentsGeneral(obs.reshape(n_realizations_ricker, n_length_ricker)))

mmd_npe = np.zeros(n_sim)
mmd_rnpe = np.zeros(n_sim)
mmd_npe_rs = np.zeros(n_sim)
for i in range(0, n_sim):
    # npe
    root_name = f"objects/NPE/ricker_final/degree=0.2_none_beta=2.0_theta=[4, 25]_num=1000/{i+1}"
    _, _, posterior_npe = load_models(root_name, device)
    post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_mmd)

    predictive_data_npe = torch.zeros(n_samples_mmd, n_length_ricker)
    for j in range(n_samples_mmd):
        predictive_data_npe[j] = simulator(post_samples_npe[j])[0]

    pred_stat_npe = torch.tensor(temporalMomentsGeneral(predictive_data_npe))

    mmd_npe[i] = float(metrics.MMD_unweighted(pred_stat_npe, obs_stat, lengthscale=1))

    # rnpe
    post_samples_rnpe = torch.tensor(read_rnpe(model="ricker", var=1, degree=0, seed=i+1, theta='[4,25]'))[:n_samples_mmd]

    predictive_data_rnpe = torch.zeros(n_samples_mmd, n_length_ricker)
    for j in range(n_samples_mmd):
        predictive_data_rnpe[j] = simulator(post_samples_rnpe[j])[0]

    pred_stat_rnpe = torch.tensor(temporalMomentsGeneral(predictive_data_rnpe))

    mmd_rnpe[i] = float(metrics.MMD_unweighted(pred_stat_rnpe, obs_stat, lengthscale=1))

    # npe_rs
    root_name = f"objects/NPE/ricker_final/degree=0.2_mmd_beta=5.0_theta=[4, 25]_num=1000/{i+1}"
    _, _, posterior_npe_rs = load_models(root_name, device)
    post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_mmd)

    predictive_data_npe_rs = torch.zeros(n_samples_mmd, n_length_ricker)
    for j in range(n_samples_mmd):
        predictive_data_npe_rs[j] = simulator(post_samples_npe_rs[j])[0]

    pred_stat_npe_rs = torch.tensor(temporalMomentsGeneral(predictive_data_npe_rs))

    mmd_npe_rs[i] = float(metrics.MMD_unweighted(pred_stat_npe_rs, obs_stat, lengthscale=1))

print(f"Ricker prior misspecification")
print(f"NPE: MMD mean={np.mean(mmd_npe[~np.isnan(mmd_npe)])}, MMD std={np.std(mmd_npe[~np.isnan(mmd_npe)])}")
print(f"RNPE: MMD mean={np.mean(mmd_rnpe[~np.isnan(mmd_rnpe)])}, MMD std={np.std(mmd_rnpe[~np.isnan(mmd_rnpe)])}")
print(f"NPE-RS: MMD mean={np.mean(mmd_npe_rs[~np.isnan(mmd_npe_rs)])}, MMD std={np.std(mmd_npe_rs[~np.isnan(mmd_npe_rs)])}")


Ricker prior misspecification
NPE: MMD mean=0.16637261207515547, MMD std=0.15120399465017192
RNPE: MMD mean=0.5352750061168536, MMD std=0.3971765945454501
NPE-RS: MMD mean=0.1665071753268997, MMD std=0.25160294471773914


## Turin 

In [61]:
n_realizations_turin = 100
n_length_turin = 801

n_samples_mmd = 1000
n_sim = 20

simulator = turin(B=4e9, Ns=801, N=1, tau0=0)

# for turin, we need new posterior
def load_models(root_name: str, device: torch.device):
    sum_net = torch.load("{root_name}/sum_net.pkl".format(root_name=root_name), map_location=device)

    density_estimator = torch.load("{root_name}/density_estimator.pkl".format(root_name=root_name), map_location=device)

    with open("{root_name}/posterior_new.pkl".format(root_name=root_name), "rb") as handle:
        posterior = CPU_Unpickler(handle).load() if device == torch.device('cpu') else pickle.load(handle)
    
    return sum_net, density_estimator, posterior

In [62]:
def temporalMomentsGeneral_turin(Y, K=4, B=4e9):
    N, Ns = Y.shape
    delta_f = B / (Ns - 1)
    t_max = 1 / delta_f
    tau = np.linspace(0, t_max, Ns)
    out = np.zeros((N, K))
    for k in range(K):
        for i in range(N):
            y = np.fft.ifft(Y[i, :])
            out[i, k] = np.trapz(tau ** (k) * (np.abs(y) ** 2), tau)
    return np.log(out)

### Turin MMD

In [ ]:
obs = torch.tensor(np.load(f"data/turin_obs.npy"))

obs_stat = torch.tensor(temporalMomentsGeneral_turin(Y=obs.reshape(n_realizations_turin, n_length_turin)))
mmd_npe = np.zeros(n_sim)
mmd_rnpe = np.zeros(n_sim)
mmd_npe_rs = np.zeros(n_sim)
for i in range(0, n_sim):
    print(i)
    # npe
    root_name = f"objects/NPE/turin_final/none_beta=2.0_num=2000_N=100_tau0/{i+1}"
    _, _, posterior_npe = load_models(root_name, device)
    post_samples_npe = sample_posteriors(posterior_npe, obs, n_samples_mmd)

    predictive_data_npe = torch.zeros(n_samples_mmd, n_length_turin)
    for j in range(n_samples_mmd):
        predictive_data_npe[j] = simulator(post_samples_npe[j])[0]

    pred_stat_npe = torch.tensor(temporalMomentsGeneral_turin(predictive_data_npe))

    mmd_npe[i] = float(metrics.MMD_unweighted(pred_stat_npe, obs_stat, lengthscale=1))

    # rnpe
    post_samples_rnpe = torch.tensor(read_rnpe(model="turin", var=1, degree=0, seed=i+1, theta='[0,0]'))[:n_samples_mmd]
    
    predictive_data_rnpe = torch.zeros(n_samples_mmd, n_length_turin)
    for j in range(n_samples_mmd):
        predictive_data_rnpe[j] = simulator(post_samples_rnpe[j])[0]

    pred_stat_rnpe = torch.tensor(temporalMomentsGeneral_turin(predictive_data_rnpe))

    mmd_rnpe[i] = float(metrics.MMD_unweighted(pred_stat_rnpe, obs_stat, lengthscale=1))

    # npe_rs
    root_name = f"objects/NPE/turin_final/mmd_beta=2.0_num=2000_N=100_tau0/{i+1}"
    _, _, posterior_npe_rs = load_models(root_name, device)
    post_samples_npe_rs = sample_posteriors(posterior_npe_rs, obs, n_samples_mmd)

    predictive_data_npe_rs = torch.zeros(n_samples_mmd, n_length_turin)
    for j in range(n_samples_mmd):
        predictive_data_npe_rs[j] = simulator(post_samples_npe_rs[j])[0]

    pred_stat_npe_rs = torch.tensor(temporalMomentsGeneral_turin(predictive_data_npe_rs))

    mmd_npe_rs[i] = float(metrics.MMD_unweighted(pred_stat_npe_rs, obs_stat, lengthscale=1))

print(f"Turin model")
print(f"NPE: MMD mean={np.mean(mmd_npe[~np.isnan(mmd_npe)])}, MMD std={np.std(mmd_npe[~np.isnan(mmd_npe)])}")
print(f"RNPE: MMD mean={np.mean(mmd_rnpe[~np.isnan(mmd_rnpe)])}, MMD std={np.std(mmd_rnpe[~np.isnan(mmd_rnpe)])}")
print(f"NPE-RS: MMD mean={np.mean(mmd_npe_rs[~np.isnan(mmd_npe_rs)])}, MMD std={np.std(mmd_npe_rs[~np.isnan(mmd_npe_rs)])}")
